In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence  import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score ,classification_report, precision_recall_fscore_support
import warnings
import os

In [163]:
df1 = pd.read_csv("case_study1.xlsx - case_study1.csv")
df2 = pd.read_csv("case_study2.xlsx - case_study2.csv")

In [164]:
df1 = df1.loc[df1['Age_Oldest_TL']!= -99999]

In [165]:
df1.shape

(51296, 26)

In [166]:
columns_to_be_removed = []

for i in df2.columns:
    if df2.loc[df2[i]== - 99999].shape[0] > 10000:
        columns_to_be_removed.append(i)

In [167]:
df2 = df2.drop(columns_to_be_removed,axis=1)

In [168]:
df2.shape

(51336, 54)

In [169]:
for i in df2.columns:
    df2 = df2.loc[df2[i] != -99999]

In [170]:
df2.shape

(42066, 54)

In [171]:
for i in df1.columns:
    for j in df2.columns:
        if i == j:
            print(i)

PROSPECTID


In [172]:
df2.shape

(42066, 54)

In [173]:
df = pd.merge(df1,df2,how='inner',left_on='PROSPECTID',right_on='PROSPECTID')

In [174]:
df.shape

(42064, 79)

In [175]:
# Check How many are categorical 
categorical_columns = []
for i in df.columns:
    if df[i].dtype == 'object':
        categorical_columns.append(i)

In [176]:
df['MARITALSTATUS'].value_counts()
df['EDUCATION'].value_counts()
df['GENDER'].value_counts()
df['last_prod_enq2'].value_counts()

last_prod_enq2
ConsumerLoan    16480
others          13653
PL               7553
CC               2195
AL               1353
HL                830
Name: count, dtype: int64

In [177]:
# Chi Square Test

for i in ['MARITALSTATUS','EDUCATION','GENDER','last_prod_enq2','first_prod_enq2']:
    chi2,pval, _,_ = chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i,'---',pval)

MARITALSTATUS --- 3.578180861038862e-233
EDUCATION --- 2.6942265249737532e-30
GENDER --- 1.907936100186563e-05
last_prod_enq2 --- 0.0
first_prod_enq2 --- 7.84997610555419e-287


### Since all the categorical features have pval <= 0.05  , we will accept all

In [178]:
numeric_columns = []
for i in df.columns:
    if df[i].dtype != 'O' and i not in ['PROSPECTID','Approved_Flag']:
        numeric_columns.append(i)

In [179]:
vif_data = df[numeric_columns]
total_columns = vif_data.shape[1]
columns_to_be_kept = []
column_index = 0

# vif_data.shape

In [180]:
total_columns

72

In [181]:
for i in range(0,total_columns):
    vif_value = variance_inflation_factor(vif_data,column_index)
    print(column_index,'---',vif_value)


    
    if vif_value <= 6:
        columns_to_be_kept.append(numeric_columns[i])
        column_index = column_index + 1
    else:
        vif_data = vif_data.drop([numeric_columns[i]],axis=1)

/home/ankanbera/Documents/Credit Risk Modelling Project/.venv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf


/home/ankanbera/Documents/Credit Risk Modelling Project/.venv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0 --- inf
0 --- 11.320180023967996
0 --- 8.363698035000336
0 --- 6.520647877790928
0 --- 5.149501618212625
1 --- 2.611111040579735


/home/ankanbera/Documents/Credit Risk Modelling Project/.venv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2 --- inf
2 --- 1788.7926256209232
2 --- 8.601028256477228
2 --- 3.8328007921530785
3 --- 6.0996533816467355
3 --- 5.5813520096427585
4 --- 1.9855843530987785


/home/ankanbera/Documents/Credit Risk Modelling Project/.venv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5 --- inf
5 --- 4.809538302819343
6 --- 23.270628983464636
6 --- 30.595522588100053
6 --- 4.3843464059655854
7 --- 3.064658415523423
8 --- 2.898639771299252
9 --- 4.377876915347322
10 --- 2.2078535836958433
11 --- 4.916914200506864
12 --- 5.214702030064725
13 --- 3.3861625024231476
14 --- 7.840583309478997
14 --- 5.255034641721438


/home/ankanbera/Documents/Credit Risk Modelling Project/.venv/lib/python3.13/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15 --- inf
15 --- 7.380634506427238
15 --- 1.421005001517573
16 --- 8.083255010190323
16 --- 1.624122752404011
17 --- 7.257811920140003
17 --- 15.59624383268298
17 --- 1.825857047132431
18 --- 1.5080839450032664
19 --- 2.172088834824577
20 --- 2.6233975535272283
21 --- 2.2959970812106176
22 --- 7.360578319196446
22 --- 2.1602387773102554
23 --- 2.8686288267891458
24 --- 6.458218003637272
24 --- 2.8474118865638265
25 --- 4.753198156284083
26 --- 16.227354755948223
26 --- 6.424377256363877
26 --- 8.887080381808687
26 --- 2.3804746142952653
27 --- 8.60951347651454
27 --- 13.06755093547673
27 --- 3.5000400566546555
28 --- 1.9087955874813773
29 --- 17.006562234161628
29 --- 10.730485153719197
29 --- 2.3538497522950275
30 --- 22.104855915136433
30 --- 2.7971639638512915
31 --- 3.424171203217696
32 --- 10.175021454450935
32 --- 6.408710354561301
32 --- 1.001151196262562
33 --- 3.069197305397274
34 --- 2.8091261600643715
35 --- 20.249538381980678
35 --- 15.864576541593774
35 --- 1.833164974053

#### 72 numerical Features
#### after VIF 39 features are left 

In [182]:
vif_data.shape

(42064, 39)

#### Check Anova for column to be kept

In [183]:
len(columns_to_be_kept)

39

In [184]:
from scipy.stats import f_oneway
columns_to_be_kept_numerical = []

for i in columns_to_be_kept:
    a = list(df[i])

    
    b = list(df['Approved_Flag'])


    group_P1 = [value for value, group in zip(a,b) if group == 'P1']
    group_P2 = [value for value, group in zip(a,b) if group == 'P2']
    group_P3 = [value for value, group in zip(a,b) if group == 'P3']
    group_P4 = [value for value, group in zip(a,b) if group == 'P4']

 

    f_statistic, p_value = f_oneway(group_P1,group_P2,group_P3,group_P4)

    if p_value <=0.05:
        columns_to_be_kept_numerical.append(i)


In [185]:
features = columns_to_be_kept_numerical + ['EDUCATION','first_prod_enq2','last_prod_enq2','GENDER','MARITALSTATUS']
df = df[features+['Approved_Flag']]

In [186]:
['EDUCATION','first_prod_enq2','last_prod_enq2','GENDER','MARITALSTATUS']

['EDUCATION', 'first_prod_enq2', 'last_prod_enq2', 'GENDER', 'MARITALSTATUS']

In [187]:
#Ordinal Features: (Education)
    #SSC - 1
    #12TH - 2
    #GRADUATE - 3
    #POST-GRADUATE - 3

# other has to be verified by business end user

df.loc[df['EDUCATION'] == 'SSC',['EDUCATION']] = 1
df.loc[df['EDUCATION'] == '12TH',['EDUCATION']] = 2
df.loc[df['EDUCATION'] == 'GRADUATE',['EDUCATION']] = 3
df.loc[df['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']] = 3
df.loc[df['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']] = 4
df.loc[df['EDUCATION'] == 'OTHERS',['EDUCATION']] = 1
df.loc[df['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']] = 3

In [188]:
df['EDUCATION'] = df['EDUCATION'].astype(int)
df['EDUCATION'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 42064 entries, 0 to 42063
Series name: EDUCATION
Non-Null Count  Dtype
--------------  -----
42064 non-null  int64
dtypes: int64(1)
memory usage: 328.8 KB


In [189]:
df_encoded = pd.get_dummies(df,columns=['MARITALSTATUS','GENDER','first_prod_enq2','last_prod_enq2'])

In [190]:
df_encoded.shape

(42064, 55)

In [191]:
for i in df_encoded.columns:
    if df_encoded[i].dtype == 'bool':
        df_encoded[i] = df_encoded[i].astype(int)


In [192]:
df_encoded.to_csv("credit_risk.csv", index=False)


In [193]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42064 non-null  float64
 1   pct_tl_closed_L6M             42064 non-null  float64
 2   Tot_TL_closed_L12M            42064 non-null  int64  
 3   pct_tl_closed_L12M            42064 non-null  float64
 4   Tot_Missed_Pmnt               42064 non-null  int64  
 5   CC_TL                         42064 non-null  int64  
 6   Home_TL                       42064 non-null  int64  
 7   PL_TL                         42064 non-null  int64  
 8   Secured_TL                    42064 non-null  int64  
 9   Unsecured_TL                  42064 non-null  int64  
 10  Other_TL                      42064 non-null  int64  
 11  Age_Oldest_TL                 42064 non-null  int64  
 12  Age_Newest_TL                 42064 non-null  int64  
 13  t

In [194]:
df_encoded.shape

(42064, 55)

## Machine  Learning Model Fitting 

In [195]:
y = df_encoded['Approved_Flag']
x = df_encoded.drop(['Approved_Flag'],axis=1)

In [196]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [197]:
y_test.shape

(8413,)

## Random Forest 

In [198]:
rf_classifier = RandomForestClassifier(n_estimators=200, random_state=42)
rf_classifier.fit(x_train,y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",200
,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.Note: This parameter is tree-specific.",'gini'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=""sqrt""The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to `""sqrt""`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",'sqrt'
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the current node, ``N_t_L`` is the number of samples in theleft child, and ``N_t_R`` is the number of samples in the right child.``N``, ``N_t``, ``N_t_R`` and ``N_t_L`` all refer to the weighted sum,if ``sample_weight`` is passed... versionadded:: 0.19",0.0
,"bootstrap bootstrap: bool, default=TrueWhether bootstrap samples are used when building trees. If False, thewhole dataset is used to build each tree.",True
,"oob_score oob_score: bool or callable, default=FalseWhether to use out-of-bag samples to estimate the generalization score.By default, :func:`~sklearn.metrics.accuracy_score` is used.Provide a callable with signature `metric

In [199]:
y_pred = rf_classifier.predict(x_test)


accuracy = accuracy_score(y_test,y_pred)
print(f'Accuracy : {accuracy: .2f}')

preision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred)

for i, v in enumerate(['P1','P2','P3','P4']):
    print(f'Class : {v}')
    print(f'precision: {preision[i]}')
    print(f'recall : {recall[i]}')
    print(f'f1_score : {f1_score[i]}')

Accuracy :  0.77
Class : P1
precision: 0.8416075650118203
recall : 0.7021696252465484
f1_score : 0.7655913978494624
Class : P2
precision: 0.7965037338764427
recall : 0.9302279484638256
f1_score : 0.8581878028709884
Class : P3
precision: 0.4470774091627172
recall : 0.21358490566037736
f1_score : 0.28907048008171604
Class : P4
precision: 0.7216890595009597
recall : 0.7308066083576288
f1_score : 0.7262192177691936


## XGBOOST 

In [200]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

In [201]:
xgb_classifier = xgb.XGBClassifier(objective='multi:softmax',num_class = 4)

In [202]:
label_encode = LabelEncoder()

y_encoded = label_encode.fit_transform(y)

In [203]:
x_train,x_test,y_train,y_test = train_test_split(x,y_encoded,test_size=0.2,random_state=42)

In [204]:
xgb_classifier.fit(x_train,y_train)
y_pred = xgb_classifier.predict(x_test)

In [205]:
accuracy = accuracy_score(y_test,y_pred)
print(f'Accuracy : {accuracy: .2f}')

preision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred)

for i, v in enumerate(['P1','P2','P3','P4']):
    print(f'Class : {v}')
    print(f'precision: {preision[i]}')
    print(f'recall : {recall[i]}')
    print(f'f1_score : {f1_score[i]}')

Accuracy :  0.78
Class : P1
precision: 0.8260405549626467
recall : 0.7633136094674556
f1_score : 0.7934392619169657
Class : P2
precision: 0.8257602862254025
recall : 0.9149653121902874
f1_score : 0.8680771039022097
Class : P3
precision: 0.4662084765177549
recall : 0.3071698113207547
f1_score : 0.370336669699727
Class : P4
precision: 0.7305034550839091
recall : 0.7191448007774538
f1_score : 0.7247796278158668


## Decision Tree

In [206]:
from sklearn.tree import DecisionTreeClassifier

In [207]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [208]:
dt_classifier = DecisionTreeClassifier(max_depth=20, min_samples_split=10)
dt_classifier.fit(x_train,y_train)

,"criterion criterion: {""gini"", ""entropy"", ""log_loss""}, default=""gini""The function to measure the quality of a split. Supported criteria are""gini"" for the Gini impurity and ""log_loss"" and ""entropy"" both for theShannon information gain, see :ref:`tree_mathematical_formulation`.",'gini'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",20
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",10
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`... note:: The search for a split does not stop until at least one valid partition of the node samples is found, even if it requires to effectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow a tree with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsamples at the curren

In [209]:
y_pred = dt_classifier.predict(x_test)

In [210]:
accuracy = accuracy_score(y_test,y_pred)
print(f'Accuracy : {accuracy: .2f}')

preision,recall,f1_score,_ = precision_recall_fscore_support(y_test,y_pred)

for i, v in enumerate(['P1','P2','P3','P4']):
    print(f'Class : {v}')
    print(f'precision: {preision[i]}')
    print(f'recall : {recall[i]}')
    print(f'f1_score : {f1_score[i]}')

Accuracy :  0.71
Class : P1
precision: 0.7224975222993062
recall : 0.7189349112426036
f1_score : 0.7207118141374197
Class : P2
precision: 0.8078416149068323
recall : 0.8249752229930625
f1_score : 0.8163185250563891
Class : P3
precision: 0.3352
recall : 0.31622641509433963
f1_score : 0.32543689320388347
Class : P4
precision: 0.6447105788423154
recall : 0.6277939747327502
f1_score : 0.6361398325947809


## Hyperparameter Tuning
#### Define Hyperparameter Grid

In [211]:
param_grid  = {
    'colsample_bytree' : [0.1,0.3,0.5,0.7,0.9],
    'learning_rate' : [0.001,0.01,0.1,1],
    'max_depth' : [3,5,8,10],
    'alpha' : [1,10,100],
    'n_estimator' : [10,50,100]
}

index = 0

answer_grid = {
    'combination': [],
    'train_accuracy ' : [],
    'test_accuracy' : [],
    'colsample_bytree': [],
    'learning_rate' : [],
    'max_depth' : [],
    'alpha' : [],
    'n_estimator' : []
}

In [212]:
# for colsample_bytree in param_grid['colsample_bytree']:
#     for learning_rate in param_grid['learning_rate']:
#         for max_depth in param_grid['max_depth']:
#             for alpha in param_grid['alpha']:
#                 for n_estimator in param_grid['n_estimator']:
#                     index = index + 1
#                     model = xgb.XGBClassifier(object ='multi:softmax',
#                                               num_class = 4,
#                                               colsample_bytree = colsample_bytree,
#                                               learning_rate = learning_rate,
#                                               max_depth = max_depth,
#                                               alpha = alpha,
#                                               n_estimator = n_estimator)
#                     y = df_encoded['Approved_Flag']
#                     x = df_encoded.drop(['Approved_Flag'],axis = 1)
#                     label_encoder = LabelEncoder()
#                     y_encoded = label_encoder.fit_transform(y)
#                     x_train,x_test,y_train,y_test = train_test_split(x,y_encoded,test_size=0.2,random_state=42)
                    
#                     model.fit(x_train,y_train)


#                     #prediction
#                     y_pred_train = model.predict(x_train)
#                     y_pred_test = model.predict(x_test)

#                     train_accuracy = accuracy_score(y_train,y_pred_train)
#                     test_accuracy  = accuracy_score(y_test,y_pred_test)


#                     answer_grid['combination'].append(index)
#                     answer_grid['train_accuracy '].append(train_accuracy)
#                     answer_grid['test_accuracy'].append(test_accuracy)
#                     answer_grid['colsample_bytree'].append(colsample_bytree)
#                     answer_grid['learning_rate'].append(learning_rate)
#                     answer_grid['max_depth'].append(max_depth)
#                     answer_grid['alpha'].append(alpha)
#                     answer_grid['n_estimator'].append(n_estimator)

#                     print(f"Combination : {index}")
#                     print(f"colsample_bytree : {colsample_bytree},learning_rate: {learning_rate}, max_depth : {max_depth}, alpha:{alpha},n_estimator:{n_estimator}")
#                     print(f"Train_accuracy: {train_accuracy : .2f}")
#                     print(f"test_accuracy: {test_accuracy: .2f}")

In [213]:
unseen_data = pd.read_excel("Unseen_Dataset.xlsx")

In [214]:
unseen_data.columns

Index(['pct_tl_open_L6M', 'pct_tl_closed_L6M', 'Tot_TL_closed_L12M',
       'pct_tl_closed_L12M', 'Tot_Missed_Pmnt', 'CC_TL', 'Home_TL', 'PL_TL',
       'Secured_TL', 'Unsecured_TL', 'Other_TL', 'Age_Oldest_TL',
       'Age_Newest_TL', 'time_since_recent_payment',
       'max_recent_level_of_deliq', 'num_deliq_6_12mts', 'num_times_60p_dpd',
       'num_std_12mts', 'num_sub', 'num_sub_6mts', 'num_sub_12mts', 'num_dbt',
       'num_dbt_12mts', 'num_lss', 'recent_level_of_deliq', 'CC_enq_L12m',
       'PL_enq_L12m', 'time_since_recent_enq', 'enq_L3m', 'NETMONTHLYINCOME',
       'Time_With_Curr_Empr', 'CC_Flag', 'PL_Flag', 'pct_PL_enq_L6m_of_ever',
       'pct_CC_enq_L6m_of_ever', 'HL_Flag', 'GL_Flag', 'MARITALSTATUS',
       'EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2'],
      dtype='object')

In [215]:
unseen_data.loc[unseen_data['EDUCATION'] == 'SSC',['EDUCATION']] = 1
unseen_data.loc[unseen_data['EDUCATION'] == '12TH',['EDUCATION']] = 2
unseen_data.loc[unseen_data['EDUCATION'] == 'GRADUATE',['EDUCATION']] = 3
unseen_data.loc[unseen_data['EDUCATION'] == 'UNDER GRADUATE',['EDUCATION']] = 3
unseen_data.loc[unseen_data['EDUCATION'] == 'POST-GRADUATE',['EDUCATION']] = 4
unseen_data.loc[unseen_data['EDUCATION'] == 'OTHERS',['EDUCATION']] = 1
unseen_data.loc[unseen_data['EDUCATION'] == 'PROFESSIONAL',['EDUCATION']] = 3

In [216]:
unseen_data.shape

(100, 42)

In [217]:
unseen_data['EDUCATION'] = unseen_data['EDUCATION'].astype(int)

In [218]:
unseen_df_encoded = pd.get_dummies(unseen_data,columns=['MARITALSTATUS','GENDER','first_prod_enq2','last_prod_enq2'])

In [219]:
unseen_df_encoded.shape

(100, 54)

In [220]:
for i in unseen_df_encoded.columns:
    if unseen_df_encoded[i].dtype == 'bool':
        unseen_df_encoded[i] = unseen_df_encoded[i].astype(int)


In [221]:
unseen_df_encoded.shape

(100, 54)

In [222]:
unseen_df_encoded

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,...,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others,last_prod_enq2_AL,last_prod_enq2_CC,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others
0,0.000,0.0,0,0.000,0,0,0,4,1,4,...,0,0,1,0,0,0,0,0,1,0
1,0.000,0.0,0,0.000,0,0,0,0,0,1,...,1,0,0,0,0,0,1,0,0,0
2,0.125,0.0,0,0.000,1,0,0,0,2,6,...,0,0,0,1,0,0,1,0,0,0
3,0.000,0.0,0,0.000,0,0,0,0,3,0,...,0,0,0,0,1,0,0,0,0,0
4,0.000,0.0,1,0.167,0,0,0,0,6,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000,0.0,0,0.000,0,0,0,1,1,5,...,0,0,0,1,0,0,1,0,0,0
96,0.222,0.0,2,0.222,0,0,0,0,1,8,...,0,0,0,1,0,0,1,0,0,0
97,0.000,0.0,0,0.000,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1
98,0.000,1.0,1,1.000,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,1


In [223]:
model_final = xgb.XGBClassifier(objective='multi:softmax',
                          num_class = 4,
                          colsample_bytree = 0.9,
                          learning_rate = 1,
                          max_depth = 3,
                          alpha = 10,
                          n_estimator = 100)

In [224]:
label_encode = LabelEncoder()

y_encoded1 = label_encode.fit_transform(y_train)

In [225]:
y_encoded1

array([0, 0, 1, ..., 2, 2, 0], shape=(33651,))

In [226]:
model_final.fit(x_train,y_encoded1)

/home/ankanbera/Documents/Credit Risk Modelling Project/.venv/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [00:46:06] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimator" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,"objective objective: typing.Union[str, xgboost.sklearn._SklObjWProto, typing.Callable[[typing.Any, typing.Any], typing.Tuple[numpy.ndarray, numpy.ndarray]], NoneType]Specify the learning task and the corresponding learning objective or a customobjective function to be used.For custom objective, see :doc:`/tutorials/custom_metric_obj` and:ref:`custom-obj-metric` for more information, along with the end note forfunction signatures.",'multi:softmax'
,"base_score base_score: typing.Union[float, typing.List[float], NoneType]The initial prediction score of all instances, global bias.",None
,booster,None
,"callbacks callbacks: typing.Optional[typing.List[xgboost.callback.TrainingCallback]]List of callback functions that are applied at end of each iteration.It is possible to use predefined callbacks by using:ref:`Callback API `... note:: States in callback are not preserved during training, which means callback objects can not be reused for multiple training sessions without reinitialization or deepcopy... code-block:: python for params in parameters_grid: # be sure to (re)initialize the callbacks before each run callbacks = [xgb.callback.LearningRateScheduler(custom_rates)] reg = xgboost.XGBRegressor(**params, callbacks=callbacks) reg.fit(X, y)",None
,colsample_bylevel colsample_bylevel: typing.Optional[float]Subsample ratio of columns for each level.,None
,colsample_bynode colsample_bynode: typing.Optional[float]Subsample ratio of columns for each split.,None
,colsample_bytree colsample_bytree: typing.Optional[float]Subsample ratio of columns when constructing each tree.,0.9
,"device device: typing.Optional[str].. versionadded:: 2.0.0Device ordinal, available options are `cpu`, `cuda`, and `gpu`.",None
,"early_stopping_rounds early_stopping_rounds: typing.Optional[int].. versionadded:: 1.6.0- Activates early stopping. Validation metric needs to improve at least once in every **early_stopping_rounds** round(s) to continue training. Requires at least one item in **eval_set** in :py:meth:`fit`.- If early stopping occurs, the model will have two additional attributes: :py:attr:`best_score` and :py:attr:`best_iteration`. These are used by the :py:meth:`predict` and :py:meth:`apply` methods to determine the optimal number of trees during inference. If users want to access the full model (including trees built after early stopping), they can specify the `iteration_range` in these inference methods. In addition, other utilities like model plotting can also use the entire model.- If you prefer to discard the trees after `best_iteration`, consider using the callback function :py:class:`xgboost.callback.EarlyStopping`.- If there's more than one item in **eval_set**, the last entry will be used for early stopping. If there's more than one metric in **eval_metric**, the last metric will be used for early stopping.",None
,enable_categorical enable_categorical: boolSee the same parameter of :py:class:`DMatrix` for details.,False
,"eval_metric eval_metric: typing.Union[str, typing.List[typing.Union[str, typing.Callable]], typing.Callable, NoneType].. versionadded:: 1.6.0Metric used for monitoring the training result and early stopping. It can be astring or list of strings as names of predefined metric in XGBoost (See:doc:`/parameter`), one of the metrics in :py:mod:`sklearn.metrics`, or anyother user defined metric that looks like `sklearn.metrics`.If custom objective is also provided, then custom metric should implement thecorresponding reverse link function.Unlike the `scoring` parameter commonly used in scikit-learn, when a callableobject is provided, it's assumed to be a cost function and by default XGBoostwill minimize the result during early stopping.For advanced usage on Early stopping like directly choosing to maximize insteadof minimize, see :py:obj:`xgboost.callback.EarlyStopping`.See :doc:`/tutorials/custom_metric_obj` and :ref:`custom-obj-metric` for moreinformation... code-block:: python from sklearn.datasets import load_diabetes fro

In [227]:
unseen_predited_value = model_final.predict(unseen_df_encoded)

In [229]:
import pickle

fileName = 'eps_v1.sav'

pickle.dump(model_final,open(fileName,'wb'))